In [1]:
# tweepy is used to call the Twitter API from Python
import tweepy
import re
 
# Authenticate to Twitter API
auth = tweepy.OAuthHandler(TWITTER_CONSUMER_KEY, TWITTER_CONSUMER_SECRET)
auth.set_access_token(TWITTER_ACCESS_TOKEN_KEY, TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

NameError: name 'TWITTER_CONSUMER_KEY' is not defined